In [1]:
import pandas as pd
iris_data = pd.read_csv("c:/data/iris/iris.csv", encoding="utf-8")
iris_data

,SepalLength,SepalWidth,PetalLength,PetalWidth,Name,Species
0,5.1,3.5,1.4,0.2,setosa,0
1,4.9,3.0,1.4,0.2,setosa,0
2,4.7,3.2,1.3,0.2,setosa,0
3,4.6,3.1,1.5,0.2,setosa,0
4,5.0,3.6,1.4,0.2,setosa,0
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica,2
146,6.3,2.5,5.0,1.9,virginica,2
147,6.5,3.0,5.2,2.0,virginica,2
148,6.2,3.4,5.4,2.3,virginica,2


In [2]:
X = iris_data.loc[:, ["SepalLength","SepalWidth","PetalLength","PetalWidth"]]
y = iris_data.loc[:,"Species"]

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
#StandardScaler() 표준화 처리 함수
#scl : 스케일러, clf : 분류모형
pipe_svc = Pipeline([('scl', StandardScaler()), ('clf', SVC(random_state=1))])
#             작업 순서:  스케일링              >               분류
param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
params = [
    {'clf__C': param_range, 'clf__kernel': ['linear']},
    #파이프라인변수명__파라미터변수명
    {'clf__C': param_range, 'clf__gamma': param_range, 'clf__kernel': ['rbf']}]

In [4]:
# cv 교차검증 횟수
gs = GridSearchCV(estimator=pipe_svc, param_grid=params, scoring='accuracy', cv=10)

In [5]:
# 시간측정
%time gs = gs.fit(X, y)
#GridSearchCV 모형
print(gs)
print(gs.best_params_)#최적의 파라미터
print(gs.best_estimator_) #최적의 모형

Wall time: 4.63 s
GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('scl', StandardScaler()),
                                       ('clf', SVC(random_state=1))]),
             param_grid=[{'clf__C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0,
                                     1000.0],
                          'clf__kernel': ['linear']},
                         {'clf__C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0,
                                     1000.0],
                          'clf__gamma': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0,
                                         100.0, 1000.0],
                          'clf__kernel': ['rbf']}],
             scoring='accuracy')
{'clf__C': 1.0, 'clf__gamma': 0.1, 'clf__kernel': 'rbf'}
Pipeline(steps=[('scl', StandardScaler()),
                ('clf', SVC(gamma=0.1, random_state=1))])


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y,random_state=10)

In [7]:
from sklearn.metrics import accuracy_score
y_pred = gs.predict(X_test)
print("최종 정확도:" , accuracy_score(y_test, y_pred))

최종 정확도: 1.0


In [8]:
#최적의 파라미터로 만든 모형
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
import numpy as np
model = SVC(random_state=1,C=10, gamma=0.01, kernel='rbf')
skf=StratifiedKFold(n_splits=10,shuffle=True,random_state=10)
#                   클래스별로 동수로 10회
result=cross_val_score(model,X,y,cv=skf)
print(result)
print(np.mean(result)*100)

[1.         1.         1.         1.         0.86666667 1.
 0.93333333 0.93333333 1.         1.        ]
97.33333333333334
